<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

---

Lets start by importing the variables of interest from our other projects.

---

In [5]:
#Takes the variables of interest from Project 3 - Part 1
%store -r house

In [8]:
#Taking a look to ensure we have the correct dataset
house.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Age,Remod_Age,Total_SF,LogSalePrice,NeighborhoodPrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,2,2008,WD,Normal,208500,7,7,2566,12.247694,197200
2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,5,2007,WD,Normal,181500,34,34,2524,12.109011,218000
3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,9,2008,WD,Normal,223500,9,8,2706,12.317167,197200



---

**Baseline**

For a classifier model, we must first look into the baseline for comparison

---


In [50]:
#Number of abnormals
num_abnorm = (house["SaleCondition"] == "Abnorml").sum()
abnorm_perc = num_abnorm/len(house["SaleCondition"])
abnorm_perc

print("The baseline percent for abnormal houses is {:.4f}%".format(abnorm_perc*100))

The baseline percent for abnormal houses is 6.5835%


In [52]:
house.groupby(by = "SaleCondition").sum()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,PoolArea,MiscVal,MoSold,YrSold,SalePrice,Age,Remod_Age,Total_SF,LogSalePrice,NeighborhoodPrice
SaleCondition,,,,,,,,,,,,,,,,,,,,,
Abnorml,5565,6465.0,897347,536,518,186265,187654,7479.0,37972,7590,...,512,1020,610,190723,13727802,4685,3296,229613,1122.189029,14841547
AdjLand,280,218.0,32010,20,19,7780,7876,250.0,1257,0,...,0,0,25,8027,416500,260,164,6809,46.116588,487000
Alloca,830,721.5,129283,60,57,21663,21810,910.0,6608,0,...,738,0,62,22088,1952536,447,300,28809,132.040511,1771098
Family,1145,1456.0,192081,116,107,39242,39529,1770.0,8932,714,...,0,0,128,40144,2992000,958,671,52224,237.393097,3017248
Normal,68260,82564.5,12513621,7149,6744,2345217,2362628,112533.0,520192,58950,...,1743,61860,7418,2391410,207594359,48693,31282,2982303,14296.630086,202716754
Partial,6090,9627.5,1359495,918,613,244629,244806,23481.0,61676,169,...,480,0,876,244883,32395118,591,414,390477,1516.303145,28584402
